# SuperScan End-to-End Test on Snowflake

**Purpose**: Test the complete SuperScan Graph RAG system on Snowflake

**Requirements**:
- Snowflake account with database access
- OpenAI API key (for embeddings)
- DeepSeek API key (for LLM extraction)

**Test Flow**:
1. Connect to Snowflake
2. Create Project
3. Create Schema
4. Process Document
5. Create Nodes & Edges
6. Test Retrieval

---

## 1. Setup & Configuration

In [ ]:
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

print("✓ Environment loaded")

In [ ]:
# Verify environment variables
required_vars = [
    'SNOWFLAKE_USER',
    'SNOWFLAKE_PASSWORD',
    'SNOWFLAKE_ACCOUNT',
    'SNOWFLAKE_DATABASE',
    'SNOWFLAKE_SCHEMA',
    'SNOWFLAKE_WAREHOUSE',
    'OPENAI_API_KEY',
    'DEEPSEEK_API_KEY'
]

missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"❌ Missing environment variables: {', '.join(missing_vars)}")
else:
    print("✓ All environment variables present")
    print(f"  Snowflake Account: {os.getenv('SNOWFLAKE_ACCOUNT')}")
    print(f"  Snowflake Database: {os.getenv('SNOWFLAKE_DATABASE')}")
    print(f"  Snowflake Warehouse: {os.getenv('SNOWFLAKE_WAREHOUSE')}")

## 2. Initialize Database Connection

In [ ]:
from graph_rag.db import init_database, test_connection

print("Initializing Snowflake database connection...")

try:
    # Test connection first
    if test_connection():
        print("✓ Snowflake connection successful")
    
    # Initialize database (create tables)
    init_database()
    print("✓ Database initialized (tables created/verified)")
    
except Exception as e:
    print(f"❌ Database initialization failed: {str(e)}")
    raise

## 3. Test VariantType Serialization

In [ ]:
from graph_rag.db import VariantType

# Test VariantType with different data structures
test_data = [
    {"type": "dict", "data": {"key": "value", "nested": {"deep": "data"}}},
    {"type": "list", "data": [1, 2, 3, "four", 5.0]},
    {"type": "config", "data": {
        "embedding_model": "text-embedding-3-small",
        "dimension": 1536,
        "custom": {"retry": 3}
    }}
]

variant_type = VariantType()

print("Testing VariantType serialization:")
for test in test_data:
    serialized = variant_type.process_bind_param(test["data"], None)
    print(f"  ✓ {test['type']}: {type(serialized).__name__}")

print("\n✓ VariantType working correctly")

## 4. Create Test Project

In [ ]:
from superscan.project_service import ProjectService
from uuid import uuid4

project_svc = ProjectService()

# Create project payload
project_name = f"test-snowflake-{str(uuid4())[:8]}"

project_payload = {
    "project_name": project_name,
    "display_name": "Snowflake Test Project",
    "description": "Testing SuperScan on Snowflake",
    "owner_id": "test-user",
    "config": {
        "default_embedding_model": "text-embedding-3-small",
        "embedding_dimension": 1536,
        "default_chunk_size": 512,
        "llm_model": "deepseek-chat"
    }
}

print(f"Creating project: {project_name}...")

try:
    project = project_svc.create_project(project_payload)
    print(f"✓ Project created: {project.project_id}")
    print(f"  Name: {project.project_name}")
    print(f"  Status: {project.status.value}")
    print(f"  Config keys: {list(project.config.keys())}")
except Exception as e:
    print(f"❌ Project creation failed: {str(e)}")
    raise

## 5. Create Schema

In [ ]:
from superscan.schema_service import SchemaService
from graph_rag.models.types import EntityType

schema_svc = SchemaService()

# Create Person schema
person_schema_payload = {
    "project_id": str(project.project_id),
    "schema_name": "Person",
    "entity_type": EntityType.NODE,
    "version": "1.0.0",
    "description": "Person entity from resume",
    "structured_attributes": [
        {
            "name": "name",
            "data_type": "string",
            "required": True,
            "description": "Person's full name"
        },
        {
            "name": "role",
            "data_type": "string",
            "required": False,
            "description": "Job role/title"
        },
        {
            "name": "skills",
            "data_type": "array",
            "required": False,
            "description": "List of skills"
        }
    ],
    "vector_config": {
        "dimension": 1536,
        "model": "text-embedding-3-small"
    }
}

print("Creating Person schema...")

try:
    person_schema = schema_svc.create_schema(person_schema_payload)
    print(f"✓ Schema created: {person_schema.schema_id}")
    print(f"  Name: {person_schema.schema_name}")
    print(f"  Type: {person_schema.entity_type.value}")
    print(f"  Attributes: {len(person_schema.structured_attributes)}")
except Exception as e:
    print(f"❌ Schema creation failed: {str(e)}")
    raise

## 6. Test Node Creation

In [ ]:
from graph_rag.models.node import Node, UnstructuredBlob, NodeMetadata
from graph_rag.db import get_session

# Create a test node
test_node = Node(
    node_name="John Doe",
    entity_type="Person",
    schema_id=person_schema.schema_id,
    project_id=project.project_id,
    structured_data={
        "name": "John Doe",
        "role": "Senior Software Engineer",
        "skills": ["Python", "Machine Learning", "GraphRAG"]
    },
    unstructured_data=[
        {
            "blob_id": "summary",
            "content": "Experienced software engineer with 10 years in ML and NLP",
            "content_type": "text/plain",
            "chunks": [],
            "language": "en"
        }
    ],
    vector=[0.1] * 1536,  # Mock embedding
    vector_model="text-embedding-3-small",
    node_metadata={
        "source_document_id": "test_resume.pdf",
        "extraction_method": "manual",
        "confidence_score": 1.0,
        "tags": ["person", "engineer"],
        "custom_metadata": {}
    }
)

print("Creating test node...")

try:
    session = get_session()
    session.add(test_node)
    session.commit()
    session.refresh(test_node)
    
    print(f"✓ Node created: {test_node.node_id}")
    print(f"  Name: {test_node.node_name}")
    print(f"  Structured data keys: {list(test_node.structured_data.keys())}")
    print(f"  Unstructured blobs: {len(test_node.unstructured_data)}")
    print(f"  Vector dimension: {len(test_node.vector)}")
    
    session.close()
except Exception as e:
    print(f"❌ Node creation failed: {str(e)}")
    if 'session' in locals():
        session.rollback()
        session.close()
    raise

## 7. Test Query & Retrieval

In [ ]:
from graph_rag.db import get_session
from graph_rag.models.node import Node
from graph_rag.models.project import Project

print("Testing query and retrieval...")

try:
    session = get_session()
    
    # Query project
    queried_project = session.query(Project).filter(
        Project.project_name == project.project_name
    ).first()
    
    if queried_project:
        print(f"✓ Project retrieved: {queried_project.project_name}")
        print(f"  Config type: {type(queried_project.config)}")
        print(f"  Config keys: {list(queried_project.config.keys())}")
    else:
        print("❌ Project not found")
    
    # Query nodes
    nodes = session.query(Node).filter(
        Node.project_id == project.project_id
    ).all()
    
    print(f"\n✓ Found {len(nodes)} node(s)")
    for node in nodes:
        print(f"  - {node.node_name} ({node.entity_type})")
        print(f"    Structured data: {node.structured_data}")
        print(f"    Metadata type: {type(node.node_metadata)}")
    
    session.close()
    
except Exception as e:
    print(f"❌ Query failed: {str(e)}")
    if 'session' in locals():
        session.close()
    raise

## 8. Test VARIANT Column Handling

In [ ]:
print("Testing VARIANT column serialization/deserialization...")

test_cases = [
    ("Project Config", queried_project.config),
    ("Project Stats", queried_project.stats),
    ("Project Tags", queried_project.tags),
    ("Node Structured Data", test_node.structured_data),
    ("Node Metadata", test_node.node_metadata),
]

all_passed = True

for name, value in test_cases:
    if value is None:
        print(f"  ⚠️  {name}: None (expected)")
    elif isinstance(value, (dict, list)):
        print(f"  ✓ {name}: {type(value).__name__} ✓")
    else:
        print(f"  ❌ {name}: Unexpected type {type(value).__name__}")
        all_passed = False

if all_passed:
    print("\n✓ All VARIANT columns handled correctly!")
else:
    print("\n❌ Some VARIANT columns had issues")

## 9. Cleanup (Optional)

In [ ]:
# Uncomment to delete test data

# from graph_rag.db import get_session
# 
# cleanup_prompt = input("Delete test data? (yes/no): ")
# 
# if cleanup_prompt.lower() == 'yes':
#     session = get_session()
#     
#     # Delete in reverse order (edges, nodes, schemas, project)
#     session.delete(test_node)
#     session.delete(person_schema)
#     session.delete(project)
#     
#     session.commit()
#     session.close()
#     
#     print("✓ Test data cleaned up")
# else:
#     print("Test data preserved")

print("\n" + "="*80)
print("🎉 SuperScan Snowflake Test Complete!")
print("="*80)

## Test Summary

This notebook tested:
- ✅ Snowflake connection
- ✅ Database initialization
- ✅ VariantType serialization
- ✅ Project creation with config
- ✅ Schema creation with attributes
- ✅ Node creation with structured/unstructured data
- ✅ Query & retrieval
- ✅ VARIANT column handling

**Next Steps**:
1. Test with real document processing
2. Test edge creation
3. Test vector search
4. Performance benchmarks